# Molecular Graph Model

MolPy uses graph theory to represent molecules for advanced operations like pattern matching (SMARTS) and atom typing.

**Why use Graph Models?**
- **Pattern Matching**: Find specific substructures (e.g., functional groups).
- **Atom Typing**: Assign force field types based on chemical environment.
- **Topology Analysis**: Detect rings, aromaticity, and connectivity.

---


## 1. Building a Molecular Graph

You can convert any `Atomistic` structure into an `igraph.Graph`.


In [ ]:
from molpy.core.atomistic import Atomistic
from molpy.typifier.adapter import build_mol_graph

# Create a simple molecule (Ethanol: C-C-O)
mol = Atomistic()
c1 = mol.def_atom(symbol="C", xyz=[0.0, 0.0, 0.0])
c2 = mol.def_atom(symbol="C", xyz=[1.5, 0.0, 0.0])
o1 = mol.def_atom(symbol="O", xyz=[2.0, 1.0, 0.0])
mol.def_bond(c1, c2)
mol.def_bond(c2, o1)

# Convert to graph
graph, vs_to_atomid, atomid_to_vs = build_mol_graph(mol)

print(f"Vertices: {graph.vcount()}, Edges: {graph.ecount()}")
print(f"Vertex attributes: {graph.vs.attributes()}")

## 2. SMARTS Patterns

SMARTS is a language for describing molecular patterns. We use `SMARTSGraph` to represent them.


In [ ]:
from molpy.typifier.graph import SMARTSGraph
from molpy.parser.smarts import SmartsParser

# Define a pattern: Oxygen connected to Carbon ([O][C])
smarts_str = "[O][C]"
parser = SmartsParser()

# Create SMARTS graph
pattern = SMARTSGraph(
    smarts_string=smarts_str, parser=parser, atomtype_name="alcohol_like"
)

print(f"Pattern: {pattern}")

## 3. Finding Matches

Use subgraph isomorphism to find where the pattern exists in your molecule.


In [ ]:
# Find matches
matches = graph.get_subisomorphisms_vf2(pattern)

print(f"Found {len(matches)} match(es)")

for i, match in enumerate(matches):
    print(f"Match {i + 1} indices: {match}")
    # Map back to atoms
    matched_atoms = [mol.entities.get_by_id(vs_to_atomid[idx]) for idx in match]
    symbols = [a.get("symbol") for a in matched_atoms]
    print(f"  Symbols: {symbols}")

## 4. Advanced: Ring Detection

The graph model automatically computes ring information.


In [ ]:
# Create a ring structure (Cyclopropane)
ring_mol = Atomistic()
a1 = ring_mol.def_atom(symbol="C", xyz=[0, 0, 0])
a2 = ring_mol.def_atom(symbol="C", xyz=[1, 0, 0])
a3 = ring_mol.def_atom(symbol="C", xyz=[0, 1, 0])
ring_mol.def_bond(a1, a2)
ring_mol.def_bond(a2, a3)
ring_mol.def_bond(a3, a1)

ring_graph, _, _ = build_mol_graph(ring_mol)

# Check 'in_ring' attribute
in_ring = ring_graph.vs["in_ring"]
print(f"Atoms in ring: {in_ring}")